In [61]:
import pandas as pd
from rdkit import Chem
import requests
from bs4 import BeautifulSoup
import numpy as np
from rdkit.Chem import PandasTools
from rdkit import RDConfig
PandasTools.RenderImagesInAllDataFrames(images=True)

In [62]:
resp=requests.get("https://www.daylight.com/dayhtml_tutorials/languages/smarts/smarts_examples.html")
soup = BeautifulSoup(resp.text)

In [ ]:
cookies = {'JSESSIONID': '2D015E8E08B8EBA48677F00A36F819EE'}
data={"out":"json",
      "pagenum":1,
      "pagesize":100,
     }

functionals={}

last_res=1
cur_res=0
while last_res>cur_res:
    resp=requests.post("https://ochem.eu/alerts/list.do",cookies=cookies,data=data)
    resp_data = resp.json()

    for en in resp_data['list']['substructure-alert']:
        smart=en["smart"]
        mol=Chem.MolFromSmarts(smart)
        if mol:
            functionals[en['name']]={"smarts":smart,"mol":mol}
    last_res=int(resp_data['list']['size'])
    cur_res=int(resp_data['list']['lastResult'])
    data["pagenum"]=int(resp_data['list']["pageNum"])+1
    print(last_res,cur_res,data["pagenum"])
len(functionals)

In [ ]:
raw_df=pd.read_csv("/media/julian/ex1ext4_1/downlaods/datasets/ZINC/substances.csv?count=all")
raw_df["mol"]=raw_df["smiles"].apply(lambda s:Chem.MolFromSmiles(s))
raw_df

In [ ]:
raw_df.drop(raw_df[raw_df["mol"].isnull()].index,inplace=True)

In [196]:
sub_df = raw_df.iloc[(np.random.random(50000)*len(raw_df)).astype(int)].copy()

In [ ]:

for i,(name,data) in enumerate(functionals.items()):
    #smart=Chem.MolFromSmarts()
    print(i,name,len(data["smarts"]))
    
    if name not in sub_df.columns:# and len(data["smarts"])<1000:
        def _hsm(m):
            return  m.HasSubstructMatch(data["mol"])
        sub_df[name] = sub_df["mol"].apply(_hsm)


In [213]:
funcs_names=np.array(list(functionals.keys()))
func_count=[0 for _ in funcs_names]
for i,k in enumerate(funcs_names):
    s=sub_df[k].sum()
    #print(k,s)
    func_count[i] = s
func_count=np.array(func_count)

funcs_names=funcs_names[np.argsort(-func_count)]
func_count=func_count[np.argsort(-func_count)]
ldf=len(sub_df)
#for i in range(len(funcs_names)):
#    print(funcs_names[i],func_count[i],100*func_count[i]/ldf)

selected_funcs_names=funcs_names[((100*func_count/ldf)<=60)&((100*func_count/ldf)>=20)]
print(len(selected_funcs_names))
selected_funcs_names

42


array(['4 - Aromatic carbon-alkane', 'sp2 hybridized carbon atoms (10)',
       'Five-membered heterocycles (LS)',
       'Aromatic heterocyclic compounds', '2 - CH2 (1 - Alkane group)',
       'Enamine', '1 - CH3 (1 - Alkane group)', 'Aromatic amines',
       'Six-membered heterocycles (LS)',
       'sp3 hybridized carbon atoms (6)',
       'Aromatic primary and secondary amines', 'Halogens',
       'Halogen derivatives (alkyl, alkenyl, aryl)', '13 - Ether',
       'Ethers', 'Five-membered heterocycles (HS)',
       'Carboxylic acid secondary amides', 'Aryl halide', 'Aryl halides',
       'Esters of aromatic alcohols and their thio and aza analogues',
       'Alkylarylethers', 'Nitrogen atoms (5)',
       'Five-membered heterocycles with one heteroatom (LS)',
       'Nitrogen atoms (7)', '12 - ACCH3 (4 - Aromatic carbon-alkane)',
       'AromaticHalogen', 'BASE', 'sp2 hybridized carbon atoms (8)',
       '13 - ACCH2 (4 - Aromatic carbon-alkane)', 'Halogenated benzene',
       'Aromati

In [254]:
lrdf=len(raw_df)
for i,name in enumerate(selected_funcs_names):
    data=functionals[name]
    #smart=Chem.MolFromSmarts()
    print(i,name,len(data["smarts"]),end="\t")
    
    if name not in raw_df.columns:# and len(data["smarts"])<1000:
        def _hsm(m):
            return  m.HasSubstructMatch(data["mol"])
        raw_df[name] = raw_df["mol"].apply(_hsm)
    print(100*raw_df[name].sum()/lrdf)


0 4 - Aromatic carbon-alkane 1933	59.457839372182335
1 sp2 hybridized carbon atoms (10) 161	56.97545500083486
2 Five-membered heterocycles (LS) 19	55.76807480380698
3 Aromatic heterocyclic compounds 9	55.20921689764568
4 2 - CH2 (1 - Alkane group) 2150	49.45349807981299
5 Enamine 14	48.03022207380197
6 1 - CH3 (1 - Alkane group) 1502	46.67022875271331
7 Aromatic amines 162	45.994823843713476
8 Six-membered heterocycles (LS) 21	43.42761729838036
9 sp3 hybridized carbon atoms (6) 59	40.52880280514276
10 Aromatic primary and secondary amines 39	38.75137752546335
11 Halogens 25	38.050592753381196
12 Halogen derivatives (alkyl, alkenyl, aryl) 37	38.039739522457836
13 13 - Ether 2580	37.22491233928869
14 Ethers 157	36.50859909834697
15 Five-membered heterocycles (HS) 38	35.2833528134914
16 Carboxylic acid secondary amides 55	34.12472866922692
17 Aryl halide 17	33.46635498413759
18 Aryl halides 12	33.46635498413759
19 Esters of aromatic alcohols and their thio and aza analogues 40	33.06895975

In [264]:
n_df=pd.DataFrame()
for l in ["smiles"]+list(selected_funcs_names):
    n_df[l] = raw_df[l]

In [268]:
n_df[selected_funcs_names] = n_df[selected_funcs_names].astype(int)

In [269]:
n_df.to_csv("functional_groups.csv",index=False)

import json
with open("functional_groups.json","w+") as f:
    f.write(json.dumps({"smarts":{name:functionals[name]["smarts"] for name in selected_funcs_names},
    "list":selected_funcs_names.tolist()
    }))

In [267]:
n_df

,smiles,4 - Aromatic carbon-alkane,sp2 hybridized carbon atoms (10),Five-membered heterocycles (LS),Aromatic heterocyclic compounds,2 - CH2 (1 - Alkane group),Enamine,1 - CH3 (1 - Alkane group),Aromatic amines,Six-membered heterocycles (LS),...,3 - CH (1 - Alkane group),"E3 - e.g., carbonates",NUC,Nitrogen atoms (4),ELEC,Five-membered heterocycles with two heteroatoms (LS),sp3 hybridized carbon atoms (2),Six-membered heterocycles (HS),sp2 hybridized carbon atoms (12),25 - CH3O (13 - Ether)
0,C=CCc1ccc(OCC(=O)N(CC)CC)c(OC)c1,True,True,False,False,False,True,True,False,False,...,False,False,False,True,False,False,False,False,False,True
1,C[C@@]1(c2ccccc2)OC(C(=O)O)=CC1=O,False,False,True,False,False,False,True,False,False,...,False,True,False,False,True,False,False,False,False,False
2,COc1cc(Cc2cnc(N)nc2N)cc(OC)c1N(C)C,True,True,False,True,False,False,False,True,True,...,False,False,True,True,False,False,True,True,True,True
3,O=C(C[S@@](=O)C(c1ccccc1)c1ccccc1)NO,True,False,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,CC[C@H]1[C@H](O)N2[C@H]3C[C@@]45c6ccccc6N(C)[C...,False,False,True,False,True,False,True,True,True,...,True,False,True,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598895,O=C1Nc2ccccc2/C1=C/c1cc(Br)c(O)c(Br)c1,False,True,True,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False
598896,CCOc1cc([C@@H]2NC(=S)NC(C)=C2C(=O)OC)cc(Br)c1OCC,True,True,False,False,False,False,True,False,True,...,False,True,False,False,True,False,True,True,False,False
598897,CCOc1cc([C@H]2NC(=S)NC(C)=C2C(=O)OC)cc(Br)c1OCC,True,True,False,False,False,False,True,False,True,...,False,True,False,False,True,False,True,True,False,False
598898,COc1ccc(Br)cc1[C@@H]1NC(=S)NC(c2ccccc2)=C1C(C)=O,True,True,False,False,False,False,False,False,True,...,False,True,False,False,True,False,True,True,False,True
